# 🧠 Building a Multi-Agent System with LangChain

In this notebook, we will build a **simple multi-agent system** using LangChain. We'll define two agents:

- A **Retriever Agent** that simulates information retrieval
- A **Summarizer Agent** that summarizes the retrieved info

We'll then orchestrate them to solve a task end-to-end.

Let's get started! 🚀


## 1. Install LangChain and OpenAI SDK

Make sure you have the required packages installed:


In [1]:
!pip install langchain openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Import Required Modules


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import Tool

## 3. Define Tool Functions

We define two dummy tools: one for retrieval and one for summarization.


In [3]:
# Dummy tool functions
def retrieve_info(input_text):
    return f"[Retrieved info about: {input_text}]"

def summarize_info(input_text):
    return f"[Summary of: {input_text}]"

## 4. Wrap Tool Functions with LangChain Tool Class


In [4]:
retrieval_tool = Tool(
    name="Retriever",
    func=retrieve_info,
    description="Retrieves relevant information based on the query."
)

summarization_tool = Tool(
    name="Summarizer",
    func=summarize_info,
    description="Summarizes the input information into concise form."
)

## 5. Define a Shared Prompt Template

This template will be used by both agents.


In [5]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

## 6. Load the Language Model


In [11]:
llm = ChatOpenAI(temperature=0, model='gpt-4o-mini')

## 7. Create Retriever and Summarizer Agents


In [14]:
retriever_agent = AgentExecutor(
    agent=create_openai_functions_agent(
        llm=llm,
        tools=[retrieval_tool],
        prompt=prompt_template
    ),
    tools=[retrieval_tool]
)

summarizer_agent = AgentExecutor(
    agent=create_openai_functions_agent(
        llm=llm,
        tools=[summarization_tool],
        prompt=prompt_template
    ),
    tools=[summarization_tool]
)

## 8. Define Orchestration Logic

This function chains the retriever and summarizer agents.


In [12]:
def multi_agent_flow(query):
    print("Retrieving info...")
    retrieved = retriever_agent.invoke({"input": query})
    
    print("Summarizing result...")
    summary = summarizer_agent.invoke({"input": retrieved["output"]})

    return summary["output"]

## 9. Run the Multi-Agent Pipeline


In [13]:
result = multi_agent_flow("Tell me about climate change impacts in 2024.")
print("\nFinal Answer:", result)

Retrieving info...
Summarizing result...

Final Answer: In 2024, climate change impacts are expected to manifest in several key areas:

1. **Temperature Increases**: More frequent and intense heatwaves, posing health risks.
2. **Extreme Weather Events**: Increased frequency of hurricanes, floods, and droughts, causing infrastructure damage and economic losses.
3. **Sea Level Rise**: Threats to coastal communities with flooding and erosion, leading to displacement.
4. **Ecosystem Disruption**: Affects biodiversity and species distributions, increasing extinction risks.
5. **Agricultural Impacts**: Affects crop yields and food security, with reduced productivity in some regions.
6. **Water Resources**: Strained by altered precipitation and evaporation, causing scarcity and competition.
7. **Health Impacts**: Includes respiratory issues and the spread of vector-borne diseases.

Mitigation and adaptation efforts are crucial to minimize these effects.
